In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
from peft import LoraConfig, get_peft_model
import logging
import os

# GPU 설정 - 명시적으로 9번 GPU만 사용하도록 설정(실제 연구 환경에 맞게 설정)
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "9"

# 설정 즉시 확인
print(f"CUDA_VISIBLE_DEVICES 설정: {os.environ['CUDA_VISIBLE_DEVICES']}")

logging.basicConfig(level=logging.INFO)

# GPU 상태 확인
print(f"CUDA available: {torch.cuda.is_available()}")
print(f"Visible devices: {os.environ.get('CUDA_VISIBLE_DEVICES', 'Not set')}")
print(f"Device count: {torch.cuda.device_count()}")
if torch.cuda.is_available():
    print(f"Current device: {torch.cuda.current_device()}")
    print(f"Device name: {torch.cuda.get_device_name(0)}")

# 사용할 장치 설정: CUDA_VISIBLE_DEVICES로 지정된 GPU의 0번 인덱스 사용
# CUDA_VISIBLE_DEVICES="9"로 설정했으므로 0번이 실제 9번 GPU가 됨
device = torch.device("cuda:0")  
logging.info(f"Using device: {device}")

# 추가 확인을 위한 코드
if torch.cuda.is_available():
    # 현재 사용 중인 GPU의 UUID 확인 (고유 식별자)
    current_device = torch.cuda.current_device()
    device_props = torch.cuda.get_device_properties(current_device)
    print(f"Using GPU: {device_props.name}")
    print(f"GPU Memory: {device_props.total_memory / 1024**3:.2f} GB")

# bfloat16 지원 여부 확인
is_bfloat16 = torch.cuda.is_bf16_supported()
logging.info(f"bfloat16 supported: {is_bfloat16}")

# 모델 설정값
max_seq_len = 1024  # 긴 문맥 추론을 위해 증가 가능
logging.info(f"Max Sequence length set to: {max_seq_len}")

lora_rank = 16  # 클수록 성능이 좋지만 속도 저하

# 로컬 경로 설정
local_model_path = "/home/qudwo9246/GRPO/AtomicGPT-gemma2-9B"
local_tokenizer_path = "/home/qudwo9246/GRPO/AtomicGPT-gemma2-9B"

# 4비트 양자화 설정
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.bfloat16,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
)

# 토크나이저 로드
try:
    tokenizer = AutoTokenizer.from_pretrained(local_tokenizer_path)
    logging.info("토크나이저 로드 완료")
except Exception as e:
    logging.error("토크나이저 로드 중 오류 발생:", exc_info=True)
    raise e

# 모델 로드
try:
    # device_map을 명시적으로 "cuda:0"으로 설정
    model = AutoModelForCausalLM.from_pretrained(
        local_model_path,
        quantization_config=bnb_config,  # 4bit 양자화 적용
        device_map="cuda:0",  # 명시적으로 cuda:0 지정
        attn_implementation="eager"
    )
    model.config.max_position_embeddings = max_seq_len
    logging.info("모델 로드 완료")
    # 모델이 어떤 장치에 로드되었는지 확인
    print(f"Model device: {next(model.parameters()).device}")
except Exception as e:
    logging.error("모델 로드 중 오류 발생:", exc_info=True)
    raise e

# LoRA 설정 적용
peft_config = LoraConfig(
    r=lora_rank,
    lora_alpha=2*lora_rank,
    target_modules=[
        "q_proj", "k_proj", "v_proj", "o_proj",
        "gate_proj", "up_proj", "down_proj",
    ],
    bias="none",
    task_type="CAUSAL_LM"
)

# PEFT 적용
try:
    model = get_peft_model(model, peft_config)
    logging.info("PEFT 적용 완료")
except Exception as e:
    logging.error("PEFT 설정 중 오류 발생:", exc_info=True)
    raise e

# 모델 정보 출력
first_layer = model.base_model.model.model.layers[0].self_attn.q_proj
print(f"First layer weight dtype: {first_layer.weight.dtype}")
print(f"Model is quantized: {model.is_quantized}")

print("모델 및 LoRA 설정 완료!")
print("Max position embeddings:", model.config.max_position_embeddings)

In [29]:
import re
from datasets import load_dataset, Dataset

# Load and prep dataset
SYSTEM_PROMPT = """
Respond in the following format:
<reasoning>
...
</reasoning>
<answer>
...
</answer>
"""

XML_COT_FORMAT = """\
<reasoning>
{reasoning}
</reasoning>
<answer>
{answer}
</answer>
"""

def extract_xml_answer(text: str) -> str:
    """XML 태그에서 답변을 추출하고, 실패하면 다른 방법으로 시도합니다."""
    try:
        # 기본 XML 추출 시도
        if "<answer>" in text and "</answer>" in text:
            answer = text.split("<answer>")[-1]
            answer = answer.split("</answer>")[0]
            return answer.strip()
        
        # "Answer:" 키워드로 시도
        elif "Answer:" in text:
            answer = text.split("Answer:")[-1].strip()
            # 다음 줄바꿈이나 문장 끝까지 추출
            if "\n" in answer:
                answer = answer.split("\n")[0]
            return answer.strip()
        
        # 마지막 문장 시도 (최후의 수단)
        sentences = re.split(r'[.!?]', text)
        # 비어있지 않은 마지막 문장들 중 숫자가 포함된 것 찾기
        for sentence in reversed(sentences):
            if sentence.strip() and re.search(r'\d+', sentence):
                return sentence.strip()
        
        # 아무것도 찾지 못했으면 빈 문자열 반환
        return ""
    except Exception:
        return ""

def extract_hash_answer(text: str) -> str | None:
    if "####" not in text:
        return None
    return text.split("####")[1].strip()

# uncomment middle messages for 1-shot prompting
def get_gsm8k_questions(split = "train") -> Dataset:
    data = load_dataset('openai/gsm8k', 'main')[split] # type: ignore
    data = data.map(lambda x: { # type: ignore
        'prompt': [
            {'role': 'system', 'content': SYSTEM_PROMPT},
            {'role': 'user', 'content': x['question']}
        ],
        'answer': extract_hash_answer(x['answer'])
    }) # type: ignore
    return data # type: ignore

dataset = get_gsm8k_questions()

# 숫자 추출 및 비교를 위한 유틸리티 함수
def extract_final_number(text: str) -> float | None:
    """텍스트에서 가장 관련성 높은 숫자를 추출합니다."""
    if not text:
        return None
    
    # 결론 문장에서 숫자 찾기 시도
    conclusion_patterns = [
        r"Therefore,.*?(\d+)",
        r"Thus,.*?(\d+)",
        r"In total,.*?(\d+)",
        r"The answer is.*?(\d+)",
        r"The result is.*?(\d+)",
        r"equals.*?(\d+)"
    ]
    
    for pattern in conclusion_patterns:
        match = re.search(pattern, text, re.IGNORECASE)
        if match:
            try:
                return float(match.group(1))
            except ValueError:
                pass
    
    # 모든 숫자 찾기
    numbers = re.findall(r'(\d+(?:\.\d+)?)', text)
    if numbers:
        try:
            # 마지막 숫자 반환 (일반적으로 최종 결과)
            return float(numbers[-1])
        except ValueError:
            pass
    
    return None

def compare_numbers(expected_str: str, actual_str: str) -> bool:
    """두 텍스트에서 숫자를 추출하여 비교합니다."""
    expected_num = extract_final_number(expected_str)
    actual_num = extract_final_number(actual_str)
    
    if expected_num is not None and actual_num is not None:
        # 소수점 이하 6자리까지 비교 (부동소수점 오차 허용)
        return abs(expected_num - actual_num) < 1e-6
    return False

# Reward functions
def correctness_reward_func(prompts, completions, answer, **kwargs) -> list[float]:
    """모델의 출력에서 정답을 추출하여 비교하고 보상을 계산합니다."""
    responses = [completion[0]['content'] for completion in completions]
    q = prompts[0][-1]['content'] if prompts and len(prompts[0]) > 0 else "N/A"
    
    rewards = []
    
    for idx, (response, expected_answer) in enumerate(zip(responses, answer)):
        # 디버깅 정보 출력 (첫 번째 샘플만)
        if idx == 0:
            print('-'*20)
            print(f"Question:\n{q}")
            print(f"Expected Answer:\n{expected_answer if expected_answer else 'N/A'}")
            print(f"Full Response:\n{response if response else 'N/A'}")
        
        # 정답이 없으면 0점
        if not expected_answer:
            rewards.append(0.0)
            continue
        
        # 숫자 정답인 경우 숫자 추출
        expected_num = extract_final_number(expected_answer)
        
        # 1. 답변 추출 알고리즘을 사용하여 답변 추출
        extracted_answer = extract_xml_answer(response)
        
        if idx == 0:
            print(f"Extracted Answer: {extracted_answer}")
        
        # 추출된 답변이 없으면 0점
        if not extracted_answer:
            rewards.append(0.0)
            if idx == 0:
                print(f"✗ No answer extracted! Reward: 0.0")
            continue
        
        # 숫자 정답인 경우
        if expected_num is not None:
            actual_num = extract_final_number(extracted_answer)
            
            if idx == 0:
                print(f"Expected Number: {expected_num}, Extracted Number: {actual_num}")
            
            # 숫자가 정확히 일치하는 경우
            if actual_num is not None and abs(actual_num - expected_num) < 1e-6:
                # 추출된 답변이 숫자만 있거나 단위만 포함된 경우 (예: "110" 또는 "110 miles")
                if re.match(r'^\s*\$?\d+(\.\d+)?\s*$', extracted_answer) or re.match(r'^\s*\d+(\.\d+)?\s*[a-zA-Z]+\s*$', extracted_answer):
                    rewards.append(2.0)  # 정확한 숫자만 있으면 2.0 보상
                    if idx == 0:
                        print(f"✓ CORRECT with exact number! Reward: 2.0")
                else:
                    rewards.append(1.5)  # 정확한 숫자가 있지만 추가 텍스트도 있으면 1.5 보상
                    if idx == 0:
                        print(f"✓ CORRECT with additional text! Reward: 1.5")
            else:
                rewards.append(0.0)  # 숫자가 일치하지 않으면 0 보상
                if idx == 0:
                    print(f"✗ INCORRECT number! Reward: 0.0")
        
        # 숫자가 아닌 정답인 경우 (텍스트 비교)
        else:
            # 정답 텍스트 정규화 (대소문자, 공백 등 무시)
            normalized_expected = expected_answer.lower().strip()
            normalized_extracted = extracted_answer.lower().strip()
            
            if idx == 0:
                print(f"Text comparison - Expected: '{normalized_expected}', Actual: '{normalized_extracted}'")
            
            # 정확히 일치하는 경우
            if normalized_expected == normalized_extracted:
                rewards.append(2.0)
                if idx == 0:
                    print(f"✓ CORRECT with exact match! Reward: 2.0")
            # 부분 일치하는 경우 (핵심 키워드가 포함된 경우)
            elif normalized_expected in normalized_extracted or normalized_extracted in normalized_expected:
                rewards.append(1.5)
                if idx == 0:
                    print(f"✓ CORRECT with partial match! Reward: 1.5")
            else:
                rewards.append(0.0)
                if idx == 0:
                    print(f"✗ INCORRECT text! Reward: 0.0")
    
    return rewards

def int_reward_func(completions, **kwargs) -> list[float]:
    """출력에 숫자가 포함되어 있으면 보상을 줍니다."""
    responses = [completion[0]['content'] for completion in completions]
    
    # 전체 응답에서 숫자 찾기
    rewards = []
    for r in responses:
        if re.search(r'\d+', r):
            rewards.append(0.5)
        else:
            rewards.append(0.0)
    
    return rewards

def strict_format_reward_func(completions, **kwargs) -> list[float]:
    """응답이 정확한 XML 형식을 따르는지 확인하고 보상을 줍니다."""
    # 정확한 XML 형식 패턴
    pattern = r"^<reasoning>\n.*?\n</reasoning>\n<answer>\n.*?\n</answer>\n$"
    responses = [completion[0]["content"] for completion in completions]
    
    rewards = []
    for r in responses:
        if re.search(pattern, r, re.DOTALL):
            rewards.append(0.5)
        else:
            rewards.append(0.0)
    
    return rewards

def soft_format_reward_func(completions, **kwargs) -> list[float]:
    """응답이 유연한 형식(XML 또는 키워드)을 따르는지 확인하고 보상을 줍니다."""
    # XML 형식 또는 키워드 형식 패턴
    pattern = r"<reasoning>.*?</reasoning>\s*<answer>.*?</answer>"
    
    responses = [completion[0]["content"] for completion in completions]
    
    rewards = []
    for r in responses:
        if re.search(pattern, r, re.DOTALL):
            rewards.append(0.25)
        else:
            rewards.append(0.0)
    
    return rewards

def count_xml(text) -> float:
    """XML 태그 또는 키워드의 존재 여부에 따라 보상을 계산합니다."""
    if not text:
        return 0.0
        
    count = 0.0
    # XML 태그 확인
    if "<reasoning>" in text:
        count += 0.125
    if "</reasoning>" in text:
        count += 0.125
    if "<answer>" in text:
        count += 0.125
    if "</answer>" in text:
        count += 0.125
    
    # 대체 형식 확인 (XML 태그가 없는 경우에도 일부 보상)
    if "Reasoning:" in text:
        count += 0.0625
    if "Answer:" in text:
        count += 0.0625
        
    return count

def xmlcount_reward_func(completions, **kwargs) -> list[float]:
    """응답에서 XML 태그 또는 키워드의 존재 여부를 측정하여 보상합니다."""
    contents = [completion[0]["content"] for completion in completions]
    return [count_xml(c) for c in contents]

In [39]:
import os
import json
import time
import logging
from datetime import datetime
import google.generativeai as genai

# 여기에 API 키 입력
GEMINI_API_KEY = ""  # 실제 사용 시 이 부분에 API 키를 넣으세요
os.environ['GEMINI_API_KEY'] = GEMINI_API_KEY
genai.configure(api_key=GEMINI_API_KEY)

# Gemini API 설정 (간소화)
def setup_gemini():
    """Gemini API 설정"""
    return genai.GenerativeModel('gemini-2.0-flash-thinking-exp')

# 로깅 설정
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s',
    handlers=[
        logging.FileHandler("gemini_evaluation.log"),
        logging.StreamHandler()
    ]
)
logger = logging.getLogger("gemini_evaluator")

# 피드백 캐시 초기화
reasoning_score_cache = {}
# 평가 결과를 저장할 전역 리스트
evaluation_history = []

def extract_reasoning(text: str) -> str:
    """<reasoning> 태그 내용을 추출합니다."""
    try:
        if "<reasoning>" in text and "</reasoning>" in text:
            reasoning = text.split("<reasoning>")[-1]
            reasoning = reasoning.split("</reasoning>")[0]
            return reasoning.strip()
        return ""
    except Exception:
        return ""

def extract_potential_reasoning(response: str) -> str:
    """
    태그가 없는 경우에도 풀이 과정으로 볼 수 있는 텍스트를 추출합니다.
    """
    # 답변 부분 추출
    answer_part = extract_xml_answer(response)
    
    # 전체 응답에서 답변 부분이나 <answer> 태그 관련 텍스트 제거
    full_text = response
    if answer_part:
        full_text = full_text.replace(f"<answer>{answer_part}</answer>", "")
    full_text = full_text.replace("<answer>", "").replace("</answer>", "")
    
    # 남은 텍스트에서 <reasoning> 태그 관련 텍스트 제거
    reasoning = full_text.replace("<reasoning>", "").replace("</reasoning>", "").strip()
    
    # 내용이 너무 짧으면 유효한 풀이 과정으로 간주하지 않음
    if len(reasoning.split()) < 10:  # 10단어 미만이면 너무 짧다고 판단
        return ""
    
    return reasoning

def generate_reasoning_score(question: str, response: str, expected_answer: str, strict_mode=False) -> tuple[float, str, dict]:
    """
    Gemini 모델을 사용하여 풀이 과정(reasoning)에 대한 점수를 생성합니다.
    
    반환값:
    - 풀이 과정 점수 (0-5)
    - 간략한 피드백
    - 상세 평가 정보 (디버깅용)
    """
    # 캐시 키 생성 (질문과 응답의 일부를 사용)
    cache_key = f"{question[:50]}_{response[:100]}_{strict_mode}"
    
    # 캐시에서 결과 확인
    if cache_key in reasoning_score_cache:
        result = reasoning_score_cache[cache_key]
        logger.debug(f"캐시에서 결과 로드: {result}")
        # 캐시에서는 점수와 피드백만 반환하므로 빈 평가 정보 추가
        return result[0], result[1], {"cached": True}
    
    # 풀이 과정 추출
    reasoning = extract_reasoning(response)
    
    # 엄격하지 않은 모드에서 태그가 없으면 다른 방법으로 풀이 과정 추출 시도
    if not reasoning and not strict_mode:
        reasoning = extract_potential_reasoning(response)
        if reasoning:
            logger.info(f"태그 없이 추출된 풀이 과정: {reasoning[:100]}...")
    
    # 풀이 과정이 없으면 0점 반환
    if not reasoning:
        logger.warning("풀이 과정이 없거나 너무 짧습니다.")
        return 0.0, "풀이 과정이 없거나 너무 짧습니다.", {"error": "풀이 과정 없음"}
    
    # Gemini 모델 초기화
    try:
        gemini_model = setup_gemini()
    except Exception as e:
        logger.error(f"Gemini API 설정 오류: {e}")
        return 0.0, f"API 오류: {str(e)}", {"error": str(e)}
    
    scoring_prompt = f"""
    당신은 수학 문제 풀이 과정을 평가하는 전문가입니다. 
    학생의 풀이 과정을 분석하고 0-5 점수로 평가해주세요.
    
    문제:
    {question}
    
    정답:
    {expected_answer}
    
    학생의 풀이 과정:
    {reasoning}
    
    다음 기준으로 풀이 과정을 평가하세요:
    - 5점: 탁월한 풀이 과정. 논리적이고 명확하며, 가장 효율적인 방법 사용
    - 4점: 매우 좋은 풀이 과정. 논리적이고 단계별로 잘 설명됨
    - 3점: 좋은 풀이 과정. 올바른 접근법이지만 설명이 다소 부족함
    - 2점: 기본적인 풀이 과정. 정답에 도달할 수 있지만 논리적 비약이 있음
    - 1점: 부족한 풀이 과정. 오류가 있거나 불완전함
    - 0점: 풀이 과정이 없거나 완전히 잘못됨
    
    JSON 형식으로 다음과 같이 응답해주세요:
    {{
        "reasoning_score": 점수(0-5),
        "brief_feedback": "한 문장으로 된 간략한 피드백",
        "detailed_evaluation": "그 문제에 핵심적으로 필요한 풀이 과정과 비교해 학생의 풀이 과정에서 핵심을 평가. (2-3문장)"
    }}
    
    다른 설명 없이 JSON만 반환해주세요. 백틱(```)이나 다른 마크다운 형식을 사용하지 마세요.
    """
    
    logger.info(f"Gemini API에 평가 요청 전송 중...")
    
    try:
        # API 호출 전 짧은 대기 (API 제한 방지)
        time.sleep(0.5)
        
        # Gemini 모델에 평가 요청
        start_time = datetime.now()
        result = gemini_model.generate_content(scoring_prompt)
        end_time = datetime.now()
        api_response_time = (end_time - start_time).total_seconds()
        
        result_text = result.text
        logger.info(f"Gemini API 응답 수신 (응답 시간: {api_response_time:.2f}초)")
        
        # 백틱(```) 제거 - JSON 파싱 오류 해결
        result_text = result_text.replace('```json', '').replace('```', '').strip()
        
        # JSON 파싱
        try:
            result_json = json.loads(result_text)
            reasoning_score = result_json.get("reasoning_score", 0)
            brief_feedback = result_json.get("brief_feedback", "")
            detailed_evaluation = result_json.get("detailed_evaluation", "")
            
            # 평가 결과 로깅
            logger.info(f"풀이 과정 평가 결과: {reasoning_score}/5")
            logger.info(f"간략 피드백: {brief_feedback}")
            logger.info(f"상세 평가: {detailed_evaluation}")
            
            # 평가 기록 저장
            evaluation_record = {
                "timestamp": datetime.now().isoformat(),
                "question": question[:100] + "..." if len(question) > 100 else question,
                "reasoning": reasoning[:100] + "..." if len(reasoning) > 100 else reasoning,
                "expected_answer": expected_answer,
                "score": reasoning_score,
                "brief_feedback": brief_feedback,
                "detailed_evaluation": detailed_evaluation,
                "response_time": api_response_time
            }
            evaluation_history.append(evaluation_record)
            
            # 결과를 캐시에 저장 (점수와 피드백만)
            result_tuple = (float(reasoning_score), brief_feedback)
            reasoning_score_cache[cache_key] = result_tuple
            
            # 평가 정보 반환 (디버깅용)
            evaluation_info = {
                "score": reasoning_score,
                "brief_feedback": brief_feedback,
                "detailed_evaluation": detailed_evaluation,
                "response_time": api_response_time
            }
            
            return float(reasoning_score), brief_feedback, evaluation_info
            
        except json.JSONDecodeError as e:
            error_msg = f"JSON 파싱 오류: {result_text} - 오류: {str(e)}"
            logger.error(error_msg)
            
            # 응급 처치: 숫자만 추출해서 점수로 사용
            import re
            score_match = re.search(r'"reasoning_score":\s*(\d+)', result_text)
            if score_match:
                try:
                    extracted_score = float(score_match.group(1))
                    logger.info(f"응급 처치: JSON 파싱 실패했지만 점수 추출 성공: {extracted_score}")
                    
                    # 피드백 추출 시도
                    feedback_match = re.search(r'"brief_feedback":\s*"([^"]+)"', result_text)
                    extracted_feedback = feedback_match.group(1) if feedback_match else "파싱 오류, 점수만 추출됨"
                    
                    # 평가 기록 저장 (부분 정보)
                    evaluation_record = {
                        "timestamp": datetime.now().isoformat(),
                        "question": question[:100] + "..." if len(question) > 100 else question,
                        "reasoning": reasoning[:100] + "..." if len(reasoning) > 100 else reasoning,
                        "expected_answer": expected_answer,
                        "score": extracted_score,
                        "brief_feedback": extracted_feedback,
                        "detailed_evaluation": "파싱 오류로 인해 추출 실패",
                        "response_time": api_response_time,
                        "parsing_error": True
                    }
                    evaluation_history.append(evaluation_record)
                    
                    # 결과를 캐시에 저장
                    result_tuple = (extracted_score, extracted_feedback)
                    reasoning_score_cache[cache_key] = result_tuple
                    
                    return extracted_score, extracted_feedback, {"error": error_msg, "partial_parse": True}
                except Exception as inner_e:
                    logger.error(f"응급 처치 실패: {inner_e}")
            
            return 0.0, "JSON 파싱 오류", {"error": error_msg, "raw_response": result_text}
            
    except Exception as e:
        error_msg = f"Gemini API 오류: {e}"
        logger.error(error_msg)
        return 0.0, f"API 오류: {str(e)}", {"error": error_msg}

def reasoning_quality_reward_func(prompts, completions, answer, **kwargs) -> list[float]:
    """
    풀이 과정(reasoning)의 품질을 평가하고 보상을 계산합니다.
    """
    responses = [completion[0]['content'] for completion in completions]
    questions = [prompt[-1]['content'] for prompt in prompts]
    
    # 현재 학습 단계 정보 (있는 경우)
    current_step = kwargs.get('current_step', 0)
    total_steps = kwargs.get('total_steps', 1000)
    strict_mode = current_step > total_steps / 2  # 학습 후반부에는 엄격한 모드 적용
    
    logger.info(f"풀이 과정 평가 시작 (현재 단계: {current_step}/{total_steps}, 엄격 모드: {strict_mode})")
    
    rewards = []
    feedbacks = []  # 디버깅 및 로깅용
    
    for idx, (question, response, expected_answer) in enumerate(zip(questions, responses, answer)):
        # 풀이 과정 점수 생성 (0-5)
        reasoning_score, brief_feedback, evaluation_info = generate_reasoning_score(
            question, response, expected_answer, strict_mode=strict_mode
        )
        
        # 점수를 0-1 범위로 정규화
        normalized_score = reasoning_score / 5.0
        
        # 디버깅 정보 (첫 번째 샘플만)
        if idx == 0:
            print('-'*20)
            print(f"풀이 과정 평가:")
            print(f"질문: {question[:100]}...")
            
            # 풀이 과정 출력
            reasoning = extract_reasoning(response)
            if not reasoning and not strict_mode:
                reasoning = extract_potential_reasoning(response)
                print(f"태그 없이 추출된 풀이 과정: {reasoning[:100]}...")
            else:
                print(f"풀이 과정: {reasoning[:100]}...")
            
            print(f"풀이 과정 점수: {reasoning_score}/5 ({normalized_score:.2f})")
            print(f"피드백: {brief_feedback}")
            
            # 캐시에서 가져온 결과가 아닌 경우 상세 평가 정보 출력
            if not evaluation_info.get("cached", False):
                print(f"상세 평가: {evaluation_info.get('detailed_evaluation', '정보 없음')}")
                print(f"API 응답 시간: {evaluation_info.get('response_time', 0):.2f}초")
        
        rewards.append(normalized_score)
        feedbacks.append(brief_feedback)
    
    return rewards

# 보상 함수 조합 예시
def combined_reward_func(prompts, completions, answer, **kwargs) -> list[float]:
    """
    여러 보상 함수의 결과를 조합하여 최종 보상을 계산합니다.
    """
    # 기존 보상 함수들
    correctness_rewards = correctness_reward_func(prompts, completions, answer, **kwargs)
    format_rewards = xmlcount_reward_func(completions, **kwargs)
    
    # 새로운 풀이 과정 평가 보상
    reasoning_rewards = reasoning_quality_reward_func(prompts, completions, answer, **kwargs)
    
    # 보상 조합 (가중치 조정 가능)
    combined_rewards = []
    for idx, (c_reward, f_reward, r_reward) in enumerate(zip(correctness_rewards, format_rewards, reasoning_rewards)):
        # 정확성에 0.6, 형식에 0.1, 풀이 과정에 0.3의 가중치 부여
        combined_reward = 0.6 * c_reward + 0.1 * f_reward + 0.3 * r_reward
        combined_rewards.append(combined_reward)
        
        # 첫 번째 샘플의 보상 내역 출력
        if idx == 0:
            print(f"보상 내역 - 정확성: {c_reward:.2f}, 형식: {f_reward:.2f}, 풀이 과정: {r_reward:.2f}")
            print(f"최종 보상: {combined_reward:.2f}")
    
    return combined_rewards


# 평가 결과 요약 및 분석 함수
def summarize_evaluations(n_recent=10):
    """최근 n개의 평가 결과를 요약하고 분석합니다."""
    if not evaluation_history:
        print("평가 기록이 없습니다.")
        return
    
    recent_evals = evaluation_history[-n_recent:] if len(evaluation_history) >= n_recent else evaluation_history
    
    print(f"\n=== 최근 {len(recent_evals)}개 평가 결과 요약 ===")
    
    # 평균 점수 계산
    avg_score = sum(float(eval_record["score"]) for eval_record in recent_evals) / len(recent_evals)
    print(f"평균 점수: {avg_score:.2f}/5")
    
    # 점수 분포
    score_distribution = {}
    for i in range(6):  # 0-5점
        score_distribution[i] = sum(1 for eval_record in recent_evals if float(eval_record["score"]) == i)
    
    print("점수 분포:")
    for score, count in score_distribution.items():
        percentage = (count / len(recent_evals)) * 100
        print(f"  {score}점: {count}개 ({percentage:.1f}%)")
    
    # 평균 응답 시간
    avg_response_time = sum(eval_record.get("response_time", 0) for eval_record in recent_evals) / len(recent_evals)
    print(f"평균 API 응답 시간: {avg_response_time:.2f}초")
    
    # 자주 언급되는 피드백 키워드 분석
    all_feedback = " ".join([eval_record["brief_feedback"] + " " + eval_record.get("detailed_evaluation", "") 
                           for eval_record in recent_evals])
    
    # 간단한 키워드 빈도 분석
    keywords = ["논리적", "명확", "효율적", "오류", "불완전", "단계별", "설명", "접근법"]
    keyword_counts = {keyword: all_feedback.lower().count(keyword) for keyword in keywords}
    
    print("\n자주 언급되는 피드백 키워드:")
    for keyword, count in sorted(keyword_counts.items(), key=lambda x: x[1], reverse=True):
        if count > 0:
            print(f"  '{keyword}': {count}회 언급")
    
    # 최근 몇 개의 상세 평가 출력
    print("\n최근 평가 상세 내용:")
    for i, eval_record in enumerate(recent_evals[-3:]):  # 최근 3개만
        print(f"\n[{i+1}] 점수: {eval_record['score']}/5")
        print(f"간략 피드백: {eval_record['brief_feedback']}")
        print(f"상세 평가: {eval_record.get('detailed_evaluation', '정보 없음')}")

def save_evaluation_results():
    """현재까지의 평가 결과를 JSON 파일로 저장합니다."""
    if not evaluation_history:
        print("저장할 평가 기록이 없습니다.")
        return None
    
    # 디렉토리 생성 (없는 경우)
    os.makedirs("evaluation_results", exist_ok=True)
    
    save_path = f"evaluation_results/eval_{datetime.now().strftime('%Y%m%d_%H%M%S')}.json"
    
    try:
        with open(save_path, "w", encoding="utf-8") as f:
            json.dump(evaluation_history, f, ensure_ascii=False, indent=2)
        
        print(f"\n평가 결과가 '{save_path}'에 저장되었습니다.")
        return save_path
    except Exception as e:
        print(f"평가 결과 저장 중 오류 발생: {e}")
        return None

# GRPO 훈련 중 주기적으로 평가 결과를 확인하는 함수
def check_evaluation_progress(current_step, total_steps):
    """훈련 중 주기적으로 평가 결과를 확인합니다."""
    # 일정 간격으로만 실행 (예: 10% 진행될 때마다)
    if total_steps == 0:  # 0으로 나누기 방지
        progress_percentage = 0
    else:
        progress_percentage = (current_step / total_steps) * 100
    
    print(f"\n=== 훈련 진행 상황: {progress_percentage:.1f}% ({current_step}/{total_steps}) ===")
    
    # 평가 결과 확인
    if evaluation_history:
        # 최근 평가 결과 요약
        recent_count = min(5, len(evaluation_history))
        print(f"최근 {recent_count}개 평가 결과:")
        
        for i, eval_record in enumerate(evaluation_history[-recent_count:]):
            print(f"[{i+1}] 점수: {eval_record.get('score', 'N/A')}/5")
            print(f"    피드백: {eval_record.get('brief_feedback', 'N/A')}")
    else:
        print("아직 평가 기록이 없습니다.")

# 테스트 함수 - 실제 평가 과정을 테스트하기 위한 함수
def test_reasoning_evaluation(question, response, expected_answer, strict_mode=False):
    """풀이 과정 평가 기능을 테스트합니다."""
    print(f"\n=== 풀이 과정 평가 테스트 ===")
    print(f"질문: {question}")
    print(f"응답: {response}")
    print(f"정답: {expected_answer}")
    print(f"엄격 모드: {strict_mode}")
    
    # 풀이 과정 추출
    reasoning = extract_reasoning(response)
    if not reasoning and not strict_mode:
        reasoning = extract_potential_reasoning(response)
    
    print(f"\n추출된 풀이 과정: {reasoning}")
    
    # 평가 실행
    score, feedback, evaluation_info = generate_reasoning_score(
        question, response, expected_answer, strict_mode=strict_mode
    )
    
    print(f"\n평가 결과:")
    print(f"점수: {score}/5")
    print(f"피드백: {feedback}")
    
    if "detailed_evaluation" in evaluation_info:
        print(f"상세 평가: {evaluation_info['detailed_evaluation']}")
    
    if "response_time" in evaluation_info:
        print(f"API 응답 시간: {evaluation_info['response_time']:.2f}초")
    
    # 캐시 테스트
    print("\n캐시 테스트 (동일한 입력으로 다시 호출)...")
    start_time = time.time()
    cached_score, cached_feedback, cached_info = generate_reasoning_score(
        question, response, expected_answer, strict_mode=strict_mode
    )
    cache_time = time.time() - start_time
    
    print(f"캐시된 점수: {cached_score}/5 (응답 시간: {cache_time:.4f}초)")
    print(f"캐시된 피드백: {cached_feedback}")
    
    return score, feedback, evaluation_info

In [ ]:
# 모델을 bfloat16으로 변환
model = model.to(torch.bfloat16)
print(f"모델 데이터 타입: {next(model.parameters()).dtype}")

In [ ]:
from transformers import TrainerCallback
from typing import Dict, Any
import os
import torch
from trl import GRPOConfig, GRPOTrainer

# bfloat16 지원 여부 확인
bf16_supported = torch.cuda.is_bf16_supported()

# 훈련 인자 설정
training_args = GRPOConfig(
    use_vllm=False, # ✅ vLLM 지원 여부 불확실 → False로 변경
    learning_rate=5e-6,
    adam_beta1=0.9,
    adam_beta2=0.99,
    weight_decay=0.1,
    warmup_ratio=0.1,
    lr_scheduler_type="cosine",
    optim="adamw_bnb_8bit",
    logging_steps=1,
    bf16=True, # ✅ bfloat16이 가능하면 사용
    fp16=False, # ✅ fp16 사용 안 함 (오버플로우 방지)
    per_device_train_batch_size=4, # ✅ batch size 감소 (OOM 방지)
    gradient_accumulation_steps=1, # ✅ grad accumulation 추가
    num_generations=4,
    max_prompt_length=256,
    max_completion_length=200,
    max_steps=2000,
    save_steps=500,
    max_grad_norm=0.1,
    report_to="none",
    output_dir="outputs",
    do_train=True,
    # 멀티 GPU 관련 설정 추가
    ddp_find_unused_parameters=False,
    dataloader_num_workers=2,
)

# 평가 결과를 트레이너 메트릭에 통합하는 콜백
class ReasoningEvaluationCallback(TrainerCallback):
    def __init__(self):
        self.step = 0
        self.reasoning_scores = []  # 풀이 과정 점수 기록
        self.total_scores = []      # 총점 기록
    
    def on_init_end(self, args, state, control, **kwargs):
        """초기화 완료 시 호출"""
        print("훈련 초기화 완료. 풀이 과정 평가 모니터링 시작...")
        return control
    
    # 메서드 시그니처 수정 - **kwargs만 사용
    def on_step_end(self, args, state, control, **kwargs):
        """각 스텝 완료 시 호출"""
        self.step += 1
        
        # 10 스텝마다 평가 결과 확인 및 로깅
        if self.step % 10 == 0:
            print(f"스텝 {self.step}/{args.max_steps} 완료")
            
            # 최근 평가 결과 수집
            if evaluation_history:
                recent_evals = evaluation_history[-10:]  # 최근 10개
                
                # 평균 점수 계산
                avg_score = sum(float(eval_record.get("score", 0)) for eval_record in recent_evals) / len(recent_evals)
                
                # 트레이너에 메트릭 로깅
                metrics = {
                    "reasoning_score": avg_score,
                    "reasoning_score_normalized": avg_score / 5.0,  # 0-1 범위로 정규화
                }
                
                # 트레이너의 log_metrics 메서드 호출
                if hasattr(kwargs.get('model', None), 'log_metrics'):
                    kwargs['model'].log_metrics("train", metrics)
                    kwargs['model'].save_metrics("train", metrics, combined=True)
                    print(f"메트릭 로깅 완료: {metrics}")
                
                # 점수 기록
                self.reasoning_scores.append(avg_score)
                
                # 간단한 요약 출력
                print(f"최근 평균 풀이 과정 점수: {avg_score:.2f}/5")
        
        return control
    
    def on_train_end(self, args, state, control, **kwargs):
        """훈련 완료 시 호출"""
        print("훈련 완료. 최종 평가 결과 기록 중...")
        
        # 전체 평가 결과 요약
        if evaluation_history:
            # 평균 점수 계산
            avg_score = sum(float(eval_record.get("score", 0)) for eval_record in evaluation_history) / len(evaluation_history)
            
            # 최종 메트릭 기록
            final_metrics = {
                "final_reasoning_score": avg_score,
                "final_reasoning_score_normalized": avg_score / 5.0,
                "num_evaluations": len(evaluation_history)
            }
            
            # 평가 결과 파일로 저장
            save_path = save_evaluation_results()
            
            # 트레이너 상태에 평가 결과 파일 경로 추가
            if save_path and hasattr(state, 'log_history'):
                state.log_history.append({
                    "reasoning_evaluation_file": save_path,
                    "step": self.step
                })
        
        return control

# 평가 결과를 직접 GRPO 트레이너에 통합하는 함수
def integrate_reasoning_evaluation(prompts, completions, answer, **kwargs) -> list[float]:
    """
    풀이 과정 평가 결과를 계산하고, 결과를 트레이너에 직접 통합합니다.
    """
    # 기존 함수와 동일하게 보상 계산
    rewards = reasoning_quality_reward_func(prompts, completions, answer, **kwargs)
    
    # 최근 평가 결과가 있으면 기록
    if evaluation_history:
        # 현재 스텝 정보 (있는 경우)
        current_step = kwargs.get('current_step', 0)
        
        # 평균 점수 계산
        recent_evals = evaluation_history[-len(rewards):]  # 현재 배치와 동일한 수의 최근 평가
        if recent_evals:
            avg_score = sum(float(eval_record.get("score", 0)) for eval_record in recent_evals) / len(recent_evals)
            
            # 로그에 기록
            print(f"현재 배치 평균 풀이 과정 점수: {avg_score:.2f}/5")
    
    return rewards

# 콜백 인스턴스 생성
evaluation_callback = EvaluationMonitorCallback()

# GRPO 트레이너 설정 - 풀이 과정 평가 보상 함수 추가
# GRPO 트레이너 설정
trainer = GRPOTrainer(
    model = model,
    processing_class = tokenizer,
    reward_funcs = [
        xmlcount_reward_func,
        soft_format_reward_func,
        strict_format_reward_func,
        int_reward_func,
        correctness_reward_func,
        # 수정된 함수 사용
        integrate_reasoning_evaluation,
    ],
    args = training_args,
    train_dataset = dataset,
    callbacks = [ReasoningEvaluationCallback()],  # 수정된 콜백 사용
)
# 학습 시작 전 테스트 (선택 사항)
if False:  # 테스트하려면 True로 변경
    # 샘플 데이터로 풀이 과정 평가 테스트
    sample_idx = 0
    sample = dataset[sample_idx]
    question = sample['prompt'][-1]['content']
    expected_answer = sample['answer']
    
    # 모델의 현재 응답 생성
    inputs = tokenizer(question, return_tensors="pt").to(model.device)
    outputs = model.generate(**inputs, max_length=512)
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    
    # 평가 테스트
    test_reasoning_evaluation(question, response, expected_answer)
    
    print("\n테스트 완료. 계속하려면 Enter를 누르세요...")
    input()

# 학습 시작
print("학습 시작...")
trainer.train()

# 학습 완료 후 평가 결과 저장
print("학습 완료. 평가 결과 저장 중...")
save_path = save_evaluation_results()
print(f"평가 결과가 '{save_path}'에 저장되었습니다.")

# 평가 결과 요약
summarize_evaluations()